### 학습 기본 과정
1. 전처리
  -1. 정규화
  -2. 원 핫 인코딩 벡터 변환
    * tf.keras.utils.to_categorical
2. 모델 구축 (Dense layer -> MLP)
3. 스케줄러 정의
4. 훈련 (by using 'model.fit') 

In [1]:
## Basic
## import
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

# train, test data 생성
(x_train,y_train),(x_test,y_test) = \
  mnist.load_data()

In [ ]:
# 1. 전처리
## -1. 정규화
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0 ## 표준편차 1.
x_test /= 255.0

In [ ]:
## -2. 원 핫 인코딩 벡터 변환
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [15]:
# 2. 모델 구축
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(units=5,activation='sigmoid'))
model.add(tf.keras.layers.Dense(units=10,activation='softmax'))
model.summary()

opt = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 5)                 3925      
                                                                 
 dense_7 (Dense)             (None, 10)                60        
                                                                 
Total params: 3,985
Trainable params: 3,985
Non-trainable params: 0
_________________________________________________________________


In [14]:
# ** 콜백설정
## ==> 특정 조건에서 모델 조기종료
callback =tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                           min_delta=0.001,#최소갱신값
                                           patience=1,     #위반허용횟수
                                           verbose=1,      #메시지 출력
                                           mode='auto')    #min:감소시 스탑
                                                           #max:증가시 스탑
## 학습
ret = model.fit(x_train,y_train,epochs=100,batch_size=200,
                validation_split=0.2,verbose=2,callbacks=[callback])

Epoch 1/100
240/240 - 1s - loss: 1.1159 - accuracy: 0.7198 - val_loss: 0.6896 - val_accuracy: 0.8134 - 701ms/epoch - 3ms/step
Epoch 2/100
240/240 - 0s - loss: 0.6116 - accuracy: 0.8300 - val_loss: 0.5084 - val_accuracy: 0.8583 - 315ms/epoch - 1ms/step
Epoch 3/100
240/240 - 0s - loss: 0.5105 - accuracy: 0.8558 - val_loss: 0.4619 - val_accuracy: 0.8712 - 336ms/epoch - 1ms/step
Epoch 4/100
240/240 - 0s - loss: 0.4778 - accuracy: 0.8636 - val_loss: 0.4391 - val_accuracy: 0.8758 - 337ms/epoch - 1ms/step
Epoch 5/100
240/240 - 0s - loss: 0.4608 - accuracy: 0.8681 - val_loss: 0.4332 - val_accuracy: 0.8770 - 347ms/epoch - 1ms/step
Epoch 6/100
240/240 - 0s - loss: 0.4477 - accuracy: 0.8716 - val_loss: 0.4400 - val_accuracy: 0.8717 - 352ms/epoch - 1ms/step
Epoch 6: early stopping


* overfitting
: loss 가 학습데이터에서는 떨어지는데,
실제에서 오히려 더 나빠지는 현상
  - why? 

In [ ]:
# 스케줄러 정의
def scheduler(epoch,lr):
  if epoch % 2 == 0 and epoch:
    return 0.1*lr
  return lr
## 2에폭 간격,학습률 0.1*learning rate로 감소.
## 변화된 학습률은 다음 에폭에 반영

callback = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)

### 텐서보드 사용
텐서보드 : 모델 정확도, 손실, 가중치, 히스토그램 등 시각화

In [2]:
# basic
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [5]:
# 전처리
## 생성
(x_train,y_train),(x_test,y_test)=\
  mnist.load_data()
## 정규화
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0

## one-hot
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [7]:
# 모델 생성 및 추가
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(units=5,activation='sigmoid'))
model.add(tf.keras.layers.Dense(units=10,activation='softmax'))

opt = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [24]:
# 텐서보드 사용
import os
path = "/Users/user/Desktop/"
if not os.path.isdir(path):
  os.mkdir(path)
log_dir = path+"3101"

callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,update_freq='epoch',
                                          histogram_freq=10,write_images=True)

ret = model.fit(x_train,y_train,epochs=100,batch_size=200,
                validation_split=0.2,verbose=2,callbacks=[callback])


Epoch 1/100
240/240 - 0s - loss: 0.3692 - accuracy: 0.8991 - val_loss: 0.4526 - val_accuracy: 0.8771 - 366ms/epoch - 2ms/step
Epoch 2/100
240/240 - 0s - loss: 0.3697 - accuracy: 0.8982 - val_loss: 0.4525 - val_accuracy: 0.8797 - 350ms/epoch - 1ms/step
Epoch 3/100
240/240 - 0s - loss: 0.3694 - accuracy: 0.8996 - val_loss: 0.4334 - val_accuracy: 0.8848 - 340ms/epoch - 1ms/step
Epoch 4/100
240/240 - 0s - loss: 0.3697 - accuracy: 0.8988 - val_loss: 0.4340 - val_accuracy: 0.8829 - 338ms/epoch - 1ms/step
Epoch 5/100
240/240 - 0s - loss: 0.3716 - accuracy: 0.8989 - val_loss: 0.4470 - val_accuracy: 0.8781 - 347ms/epoch - 1ms/step
Epoch 6/100
240/240 - 0s - loss: 0.3692 - accuracy: 0.8984 - val_loss: 0.4442 - val_accuracy: 0.8802 - 334ms/epoch - 1ms/step
Epoch 7/100
240/240 - 0s - loss: 0.3688 - accuracy: 0.8982 - val_loss: 0.4424 - val_accuracy: 0.8786 - 331ms/epoch - 1ms/step
Epoch 8/100
240/240 - 0s - loss: 0.3688 - accuracy: 0.8993 - val_loss: 0.4420 - val_accuracy: 0.8809 - 333ms/epoch - 1

In [33]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [32]:
%reload_ext tensorboard